# La représentation vectorielle des textes

Après une introduction à la notion d’espace vectoriel qui a permis de poser un cadre formel pour manipuler des vecteurs grâce à des opérations mathématiques, nous allons nous intéresser à des méthodes qui entrent dans la définition du modèle sémantique vectoriel pour représenter des documents sous une forme numérique.

## Constituer un sac de mots

La technique du sac de mots est un moyen élégant pour inscrire des documents dans un même espace vectoriel, les mots figurant ici les dimensions de l’espace. Si elle inclut toujours une étape de tokenisation, la suite dépendra souvent de l’objectif ou de choix scientifiques assumés. Dans certains cas, on considérera les lemmes ; dans d’autres, les mots-formes uniques, les racines, seulement les entités nommées ou les adjectifs, et peut-être encore ne conserverait-on pas uniquement des *n*-grammes ?

Plus qu’une simple technique, le sac de mots devient un véritable modèle dont les choix devront être étayés par des arguments scientifiques voire validés par des méthodes d’évaluation.

### Étape 1 : tokenisation du texte

Tokeniser un texte, c’est le segmenter en plusieurs parties (tokens) sans que l’unité ne soit imposée. En TALN, il est fréquent de rencontrer le mot comme unité de la segmentation. Et si là encore il existe plusieurs méthodes, nous en retiendrons une avec la bibliothèque NLTK. Prenons comme exemple le texte de *Salammbô* de Gustave Flauebrt :

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer

# textual content of the file
with open('./data/salammbo.txt') as f:
    text = f.read()

# a tokenizer based on a regular expression
tokenizer = RegexpTokenizer('\w+')

# processing…
tokens = tokenizer.tokenize(text)

# save in a file, one word a line
with open('./data/salammbo_tokenized.txt', 'w') as f:
    for token in tokens:
        f.write(token)
        f.write('\n')

### Étape 2 : étiquetage et lemmatisation des tokens

NLTK ne fournissant pas de support pour l’étiquetage en parties du discours pour le français, nous choisissons une option fournie par *TreeTagger* qui a l’avantage de lemmatiser dans la foulée :

In [ ]:
! cat ./data/salammbo_tokenized.txt | ./TreeTagger/tree-tagger-macos -token -lemma ./TreeTagger/french.par > ./data/salammbo_tagged.tsv

### Étape 3 : filtrer les tokens

Il est rare que l’on souhaite conserver tous les tokens après la phase de segmentation. Rien que dans les dix premiers de la liste, nous remarquons des éléments que nous ne souhaitons pas :

- Les chiffres ;
- les majuscules ;
- les mots vides de sens.

La liste des exclusions n’est d’une part pas exhaustive et, d’autre part, il peut se justifier de ne pas retenir l’une ou l’autre des options. Appliquons malgré tout nos choix à la liste des tokens :

In [ ]:
import csv
import string
from nltk.corpus import stopwords

with open('./data/salammbo_tagged.tsv', newline='', encoding='utf-8') as inputfile, \
open('./data/salammbo_filtered.tsv', 'w', newline='', encoding='utf-8') as outputfile:

    # fields
    fieldnames = ['token', 'tag', 'lemma']

    # files
    reader = csv.DictReader(inputfile, fieldnames=fieldnames, delimiter='\t')
    writer = csv.DictWriter(outputfile, fieldnames=fieldnames, delimiter='\t')

    # filtered tokens
    filtered_rows = [
        row for row in reader
        if row['token'].lower() not in stopwords.words('french')
        and row['token'] not in string.punctuation
        and not any(char in string.digits for char in row['token'])
    ]

    # writing in file
    writer.writerows(filtered_rows)

### Étape 4 : nettoyage des entrées

D’autres traitements peuvent encore être appliqués après toutes ces étapes. Corrigeons juste les entrées qui répondent à ces critères : si l’étiquette est à `NAM` et le lemme à `<unknown>`, alors considérons que le lemme prend la valeur du token.

In [ ]:
with open('./data/salammbo_filtered.tsv', newline='', encoding='utf-8') as inputfile, \
open('./data/salammbo_clean.tsv', 'w', newline='', encoding='utf-8') as outputfile:

    fieldnames = ['token', 'tag', 'lemma']

    reader = csv.DictReader(inputfile, fieldnames=fieldnames, delimiter='\t')
    writer = csv.DictWriter(outputfile, fieldnames=fieldnames, delimiter='\t')

    for row in reader:
        if row['lemma'] == '<unknown>' and row['tag'] == 'NAM':
            row['lemma'] = row['token']
        writer.writerow(row)

### Étape 5 : constituer le sac de mots

La phase de pré-traitement achevée, nous pouvons à présent constituer notre sac de mots afin de définir l’espace vectoriel de notre modèle :

In [ ]:
with open('./data/salammbo_clean.tsv', newline='', encoding='utf-8') as csvfile:

    fieldnames = ['token', 'tag', 'lemma']

    reader = csv.DictReader(csvfile, fieldnames=fieldnames, delimiter='\t')

    lemmas = [
        row['lemma']
        for row in reader
        if row['lemma'] != '<unknown>'
    ]

    BoW = sorted(set(lemmas))

## Établir une matrice d’occurrences

Le sac de mots obtenu représente le vocabulaire de notre corpus (constitué en l’occurrence d’un seul document). Si l’on interroge la cardinalité de l’objet `BoW`, nous remarquons immédiatement que nous évoluons dans un espace à très haute dimensionnalité :

In [ ]:
display(len(BoW))

### Étape 1 : un dictionnaire des fréquences

Plutôt que de conserver `BoW` en l’état, nous souhaitons maintenant obtenir un dictionnaire des fréquences de chaque lemme dans le texte :

In [ ]:
from collections import Counter

occurrences = Counter(lemmas)

### Étape 2 : bienvenue dans la matrice

Il ne reste plus qu’à instancier une matrice des occurrences avec *Numpy* :

In [ ]:
import numpy as np

# a null matrix
matrix = np.zeros((1, len(BoW)))

# fill the occurrences matrix
for i, lemma in enumerate(BoW):
    matrix[0][i] = occurrences[lemma]

Pour plus de clarté, on peut la visualiser dans *Pandas* :

In [ ]:
import pandas as pd

df = pd.DataFrame(matrix, columns=BoW)

display(df)

### Intuition du fléau de la dimension

En affichant la matrice d’occurrences, on se rend mieux compte de l’ampleur du phénomène : un espace en 5798 dimensions, c’est énorme ! D’autant que la plupart de ces dimensions ont des valeurs très faibles. Et encore, nous n’avons vectorisé qu’un seul document, ce qui nous assure que tous les mots du vocabulaire apparaissent au moins une fois.

Admettons avant de poursuivre que nous rajoutons un document qui comporte une seule phrase :

```txt
(1) Le petit chat boit du lait.
```

En lui appliquant le même traitement que pour le texte de *Salammbô*, nous le réduisons aux mots : *petit*, *chat*, *boire* et *lait*. Nous nous assurons qu’ils appartiennent au vocabulaire de l’œuvre et pouvons maintenant le définir comme le **vocabulaire commun** aux deux documents :

In [ ]:
print(
    "Occurrences des mots du vocabulaire du document (1) dans le document (0) :",
    f"petit : {df.petit.values[0]}",
    f"chat : {df.chat.values[0]}",
    f"boire : {df.boire.values[0]}",
    f"lait : {df.lait.values[0]}",
    sep="\n"
)

Qu’en est-il des occurrences du vocabulaire de *Salammbô* dans le document (1) ? Sur 5798 dimensions, 5794 sont fixées avec la valeur 0 :

In [ ]:
# a new document with only zeros
df.loc[len(df)] = np.zeros(len(BoW))

# data entered manually
df.loc[1].petit = 1
df.loc[1].chat = 1
df.loc[1].boire = 1
df.loc[1].lait = 1

### Des matrices creuses

On parle alors de **matrice creuse** (ou *sparse matrix* en anglais). La taille de la matrice peut déjà poser problème aux algorithmes d’apprentissage, et sa sparsité, en augmentant la complexité du calcul, peut également accroître les risques d’instabilité. On peut conseiller dans ces cas d’utiliser des structures plus adaptées comme **CSR** (*Compressed Sparse Row*) ou **CSC** (*Compressed Sparse Column*) dans `scipy.sparse` qui stockent les éléments non nuls de manière comprimée pour économiser de la mémoire.

In [ ]:
from scipy.sparse import csr_matrix

# a compressed sparse matrix
sparse_matrix = csr_matrix(df.values)

### Évolution vers d’autres méthodes

La matrice d’occurrences n’est pas une finalité du modèle vectoriel – elle ne rend compte que de fréquences brutes sans juger de leur importance relative – mais elle amène à un autre concept qui sera développé plus tard, celui de la pondération TF-IDF.

Maintenant que nous avons été confronté·e aux limites des matrices d’occurrences simples, nous comprenons mieux en quoi il est primordial de bien travailler son modèle de sac de mots en fonction de ses objectifs : plus on contrôle la taille du vocabulaire, mieux les outils mathématiques que nous mobiliserons par la suite se comporteront.